`abcnews-small.csv` is a ~10k subset of the original 1 million dataset that can be downloaded at https://www.kaggle.com/therohk/million-headlines, which is about 60m unzipped. 

Running the whole dataset on a 2017 MacBook Pro:

- the preprocessing step is about 3 minutes: `processed_docs = documents['headline_text'].map(preprocess)`
- the training step using bag of words is about 5min 46s: `lda_model = gensim.models.LdaMulticore(bow_corpus, num_topics=10, id2word=dictionary, passes=2, workers=2)`
- the training step using tf-idf is about 9min 21s: `lda_model_tfidf = gensim.models.LdaMulticore(corpus_tfidf, num_topics=10, id2word=dictionary, passes=2, workers=4)`

In [1]:
import pandas as pd

data = pd.read_csv('data/abcnews-small.csv', error_bad_lines=False);
#data = pd.read_csv('data/abcnews-date-text.csv', error_bad_lines=False);  # full dataset
data_text = data[['headline_text']]
data_text['index'] = data_text.index
documents = data_text

In [2]:
len(documents)

10001

In [3]:
documents[:5]

,headline_text,index
0,report highlights container terminal potential,0
1,mud crab business on the move,1
2,sporting task force planning begins,2
3,ama airs hospital reform concerns,3
4,health service speaks out over patient death,4


### Data Preprocessing

In [4]:
import gensim
from gensim.utils import simple_preprocess
from gensim.parsing.preprocessing import STOPWORDS
from nltk.stem import WordNetLemmatizer, SnowballStemmer
from nltk.stem.porter import *
import numpy as np
np.random.seed(42)

In [5]:
import nltk
nltk.download('wordnet')

[nltk_data] Downloading package wordnet to
[nltk_data]     /Users/harrywang/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


True

#### Lemmatize example

In [6]:
print(WordNetLemmatizer().lemmatize('went', pos='v'))

go


#### Stemmer Example

In [7]:
stemmer = SnowballStemmer('english')
original_words = ['caresses', 'flies', 'dies', 'mules', 'denied','died', 'agreed', 'owned', 
           'humbled', 'sized','meeting', 'stating', 'siezing', 'itemization','sensational', 
           'traditional', 'reference', 'colonizer','plotted']
singles = [stemmer.stem(plural) for plural in original_words]
pd.DataFrame(data = {'original word': original_words, 'stemmed': singles})

,original word,stemmed
0,caresses,caress
1,flies,fli
2,dies,die
3,mules,mule
4,denied,deni
5,died,die
6,agreed,agre
7,owned,own
8,humbled,humbl
9,sized,size


In [8]:
def lemmatize_stemming(text):
    return stemmer.stem(WordNetLemmatizer().lemmatize(text, pos='v'))

def preprocess(text):
    result = []
    for token in gensim.utils.simple_preprocess(text):
        if token not in gensim.parsing.preprocessing.STOPWORDS and len(token) > 3:
            result.append(lemmatize_stemming(token))
    return result

In [9]:
doc_sample = documents[documents['index'] == 4310].values[0][0]

print('original document: ')
words = []
for word in doc_sample.split(' '):
    words.append(word)
print(words)
print('\n\n tokenized and lemmatized document: ')
print(preprocess(doc_sample))

original document: 
['charity', 'in', 'demand', 'from', 'sacked', 'meatworkers']


 tokenized and lemmatized document: 
['chariti', 'demand', 'sack', 'meatwork']


In [10]:
%%time
# for the 1 million dataset, wall time: 2min 59s
processed_docs = documents['headline_text'].map(preprocess)

CPU times: user 1.4 s, sys: 28.6 ms, total: 1.43 s
Wall time: 1.9 s


In [11]:
processed_docs[:10]

0    [report, highlight, contain, termin, potenti]
1                                     [crab, busi]
2                 [sport, task, forc, plan, begin]
3                   [air, hospit, reform, concern]
4          [health, servic, speak, patient, death]
5                              [hors, ride, enact]
6                 [plan, intersect, revamp, begin]
7                      [stormer, slaughter, shark]
8               [england, deserv, favourit, oneil]
9      [firefight, continu, contain, effort, near]
Name: headline_text, dtype: object

### Bag of words on the dataset

In [12]:
dictionary = gensim.corpora.Dictionary(processed_docs)

In [13]:
count = 0
for k, v in dictionary.iteritems():
    print(k, v)
    count += 1
    if count > 10:
        break

0 contain
1 highlight
2 potenti
3 report
4 termin
5 busi
6 crab
7 begin
8 forc
9 plan
10 sport


In [14]:
dictionary.filter_extremes(no_below=15, no_above=0.5, keep_n=100000)

In [15]:
bow_corpus = [dictionary.doc2bow(doc) for doc in processed_docs]
bow_corpus[4310]

[(277, 1), (435, 1)]

In [16]:
bow_doc_4310 = bow_corpus[4310]

for i in range(len(bow_doc_4310)):
    print("Word {} (\"{}\") appears {} time.".format(bow_doc_4310[i][0], 
                                                     dictionary[bow_doc_4310[i][0]], 
                                                     bow_doc_4310[i][1]))

Word 277 ("demand") appears 1 time.
Word 435 ("sack") appears 1 time.


### TF-IDF

In [17]:
from gensim import corpora, models

tfidf = models.TfidfModel(bow_corpus)

In [18]:
corpus_tfidf = tfidf[bow_corpus]

In [19]:
from pprint import pprint

for doc in corpus_tfidf:
    pprint(doc)
    break

[(0, 0.802436033281265), (1, 0.5967381439893285)]


### Running LDA using Bag of Words

In [20]:
%%time
lda_model = gensim.models.LdaMulticore(bow_corpus, num_topics=10, id2word=dictionary, passes=2, workers=2)

CPU times: user 1.4 s, sys: 390 ms, total: 1.79 s
Wall time: 4.78 s


In [21]:
for idx, topic in lda_model.print_topics(-1):
    print('Topic: {} \nWords: {}'.format(idx, topic))

Topic: 0 
Words: 0.037*"interview" + 0.035*"polic" + 0.020*"home" + 0.015*"hous" + 0.014*"school" + 0.013*"brisban" + 0.012*"question" + 0.011*"plead" + 0.010*"mark" + 0.010*"sydney"
Topic: 1 
Words: 0.022*"kill" + 0.020*"protest" + 0.019*"polic" + 0.019*"chang" + 0.015*"budget" + 0.014*"trump" + 0.013*"island" + 0.013*"tasmanian" + 0.013*"speak" + 0.011*"urg"
Topic: 2 
Words: 0.040*"australia" + 0.026*"year" + 0.019*"elect" + 0.016*"servic" + 0.015*"bank" + 0.014*"council" + 0.013*"miss" + 0.012*"vote" + 0.012*"plan" + 0.011*"high"
Topic: 3 
Words: 0.013*"plan" + 0.013*"govern" + 0.013*"court" + 0.013*"farmer" + 0.012*"public" + 0.012*"turnbul" + 0.012*"million" + 0.011*"campaign" + 0.011*"hospit" + 0.011*"climat"
Topic: 4 
Words: 0.022*"open" + 0.013*"elect" + 0.013*"local" + 0.012*"make" + 0.012*"trump" + 0.012*"adelaid" + 0.012*"plan" + 0.012*"donald" + 0.011*"defend" + 0.011*"report"
Topic: 5 
Words: 0.020*"work" + 0.017*"talk" + 0.016*"melbourn" + 0.015*"help" + 0.015*"arrest" + 

Cool! Can you distinguish different topics using the words in each topic and their corresponding weights?

### Running LDA using TF-IDF

In [22]:
%%time
lda_model_tfidf = gensim.models.LdaMulticore(corpus_tfidf, num_topics=10, id2word=dictionary, passes=2, workers=4)

CPU times: user 1.43 s, sys: 177 ms, total: 1.61 s
Wall time: 3.88 s


In [23]:
for idx, topic in lda_model_tfidf.print_topics(-1):
    print('Topic: {} Word: {}'.format(idx, topic))

Topic: 0 Word: 0.020*"win" + 0.017*"australian" + 0.017*"tasmania" + 0.015*"test" + 0.013*"research" + 0.013*"look" + 0.012*"cricket" + 0.011*"launch" + 0.010*"port" + 0.010*"food"
Topic: 1 Word: 0.026*"interview" + 0.020*"say" + 0.019*"woman" + 0.016*"charg" + 0.015*"final" + 0.012*"go" + 0.012*"perth" + 0.009*"polic" + 0.009*"australian" + 0.008*"indigen"
Topic: 2 Word: 0.019*"say" + 0.016*"market" + 0.016*"polic" + 0.014*"time" + 0.013*"news" + 0.012*"brisban" + 0.011*"work" + 0.011*"melbourn" + 0.011*"monday" + 0.010*"report"
Topic: 3 Word: 0.021*"elect" + 0.017*"miss" + 0.016*"plan" + 0.012*"talk" + 0.012*"centr" + 0.012*"leader" + 0.011*"aborigin" + 0.010*"violenc" + 0.010*"white" + 0.010*"sydney"
Topic: 4 Word: 0.024*"interview" + 0.020*"kill" + 0.014*"attack" + 0.013*"coast" + 0.012*"power" + 0.011*"releas" + 0.011*"say" + 0.010*"peopl" + 0.010*"action" + 0.009*"step"
Topic: 5 Word: 0.017*"school" + 0.015*"stand" + 0.014*"hour" + 0.013*"trial" + 0.013*"fiji" + 0.012*"court" + 0

### Classification of the topics

### Performance evaluation by classifying sample document using LDA Bag of Words model

In [24]:
processed_docs[4310]

['chariti', 'demand', 'sack', 'meatwork']

In [25]:
for index, score in sorted(lda_model[bow_corpus[4310]], key=lambda tup: -1*tup[1]):
    print("\nScore: {}\t \nTopic: {}".format(score, lda_model.print_topic(index, 10)))


Score: 0.6999604105949402	 
Topic: 0.018*"dead" + 0.013*"go" + 0.013*"claim" + 0.013*"lose" + 0.011*"industri" + 0.010*"crash" + 0.010*"award" + 0.009*"region" + 0.009*"jail" + 0.009*"rule"

Score: 0.033347394317388535	 
Topic: 0.013*"plan" + 0.013*"govern" + 0.013*"court" + 0.013*"farmer" + 0.012*"public" + 0.012*"turnbul" + 0.012*"million" + 0.011*"campaign" + 0.011*"hospit" + 0.011*"climat"

Score: 0.03334241360425949	 
Topic: 0.040*"australia" + 0.026*"year" + 0.019*"elect" + 0.016*"servic" + 0.015*"bank" + 0.014*"council" + 0.013*"miss" + 0.012*"vote" + 0.012*"plan" + 0.011*"high"

Score: 0.033337824046611786	 
Topic: 0.037*"say" + 0.026*"attack" + 0.016*"crash" + 0.015*"health" + 0.014*"state" + 0.012*"call" + 0.012*"face" + 0.012*"woman" + 0.011*"report" + 0.011*"coast"

Score: 0.03333580866456032	 
Topic: 0.022*"open" + 0.013*"elect" + 0.013*"local" + 0.012*"make" + 0.012*"trump" + 0.012*"adelaid" + 0.012*"plan" + 0.012*"donald" + 0.011*"defend" + 0.011*"report"

Score: 0.0333

Our test document has the highest probability to be part of the topic on the top.

### Performance evaluation by classifying sample document using LDA TF-IDF model

In [26]:
for index, score in sorted(lda_model_tfidf[bow_corpus[4310]], key=lambda tup: -1*tup[1]):
    print("\nScore: {}\t \nTopic: {}".format(score, lda_model_tfidf.print_topic(index, 10)))


Score: 0.6999433040618896	 
Topic: 0.024*"interview" + 0.020*"kill" + 0.014*"attack" + 0.013*"coast" + 0.012*"power" + 0.011*"releas" + 0.011*"say" + 0.010*"peopl" + 0.010*"action" + 0.009*"step"

Score: 0.0333525687456131	 
Topic: 0.018*"govern" + 0.016*"want" + 0.012*"group" + 0.012*"child" + 0.010*"fund" + 0.010*"abus" + 0.010*"train" + 0.010*"sentenc" + 0.010*"year" + 0.009*"call"

Score: 0.03334299474954605	 
Topic: 0.026*"interview" + 0.020*"say" + 0.019*"woman" + 0.016*"charg" + 0.015*"final" + 0.012*"go" + 0.012*"perth" + 0.009*"polic" + 0.009*"australian" + 0.008*"indigen"

Score: 0.033339973539114	 
Topic: 0.020*"trump" + 0.018*"death" + 0.011*"make" + 0.011*"hospit" + 0.011*"australian" + 0.010*"donald" + 0.010*"perth" + 0.010*"tuesday" + 0.009*"water" + 0.009*"royal"

Score: 0.033338554203510284	 
Topic: 0.019*"crash" + 0.016*"world" + 0.012*"adelaid" + 0.011*"record" + 0.011*"farm" + 0.010*"chines" + 0.010*"north" + 0.009*"queensland" + 0.009*"canberra" + 0.009*"discuss"


Our test document has the highest probability to be part of the topic on the top.

### Testing model on unseen document

In [27]:
unseen_document = 'How a Pentagon deal became an identity crisis for Google'
bow_vector = dictionary.doc2bow(preprocess(unseen_document))

for index, score in sorted(lda_model[bow_vector], key=lambda tup: -1*tup[1]):
    print("Score: {}\t Topic: {}".format(score, lda_model.print_topic(index, 5)))

Score: 0.6999323964118958	 Topic: 0.020*"work" + 0.017*"talk" + 0.016*"melbourn" + 0.015*"help" + 0.015*"arrest"
Score: 0.03335041552782059	 Topic: 0.013*"plan" + 0.013*"govern" + 0.013*"court" + 0.013*"farmer" + 0.012*"public"
Score: 0.03334531560540199	 Topic: 0.037*"say" + 0.026*"attack" + 0.016*"crash" + 0.015*"health" + 0.014*"state"
Score: 0.0333448089659214	 Topic: 0.040*"australia" + 0.026*"year" + 0.019*"elect" + 0.016*"servic" + 0.015*"bank"
Score: 0.033340469002723694	 Topic: 0.022*"open" + 0.013*"elect" + 0.013*"local" + 0.012*"make" + 0.012*"trump"
Score: 0.03334016725420952	 Topic: 0.027*"charg" + 0.022*"australian" + 0.019*"warn" + 0.018*"face" + 0.017*"polic"
Score: 0.0333387665450573	 Topic: 0.018*"dead" + 0.013*"go" + 0.013*"claim" + 0.013*"lose" + 0.011*"industri"
Score: 0.03333800658583641	 Topic: 0.027*"say" + 0.019*"polic" + 0.017*"death" + 0.015*"group" + 0.014*"australia"
Score: 0.03333493694663048	 Topic: 0.037*"interview" + 0.035*"polic" + 0.020*"home" + 0.015

In [ ]:
%%time
# random choose 10,000 about 1/100 from the whole dataset
# no need to run this
import random

n = 1186018 # 1 million total 
s = 10000 # random 10k
skip = sorted(random.sample(range(n), n-s))
data_small = pd.read_csv('data/abcnews-date-text.csv', skiprows=skip, error_bad_lines=False, names=['index', 'headline_text'])
data_small.drop('index', axis=1, inplace=True)
data_small.to_csv('data/abcnews-small.csv')